Cross-walidacja (cross-validation) to metoda oceny modelu polegająca na **wielokrotnym dzieleniu danych na część treningową i testową**, tak aby model był sprawdzany na różnych fragmentach danych. Dzięki temu wynik oceny jest bardziej wiarygodny niż w pojedynczym train/test split.

---

# 🚀 Jak to działa — najprościej

Załóżmy, że masz zbiór danych i chcesz sprawdzić, jak dobrze działa Twój model. Zamiast:

* trenować model na 80% danych,
* testować go na pozostałych 20%,

robisz to **kilka razy na różnych podziałach**.

---

# 🔁 K-fold cross-validation (najpopularniejsza forma)

1. **Dzielisz dane na K równych części (foldów)** — np. K = 5.
2. **W każdej iteracji**:

   * bierzesz jeden fold jako *testowy*,
   * pozostałe K-1 foldów są *treningowe*,
   * trenujesz model, mierzysz wynik.
3. Powtarzasz K razy, aż każdy fold pełnił rolę danych testowych.
4. **Średnia z K wyników** jest oceną modelu.

---

📦 **Przykład przy K=5**

| Iteracja | Trening      | Test   |
| -------- | ------------ | ------ |
| 1        | fold 2,3,4,5 | fold 1 |
| 2        | fold 1,3,4,5 | fold 2 |
| 3        | fold 1,2,4,5 | fold 3 |
| 4        | fold 1,2,3,5 | fold 4 |
| 5        | fold 1,2,3,4 | fold 5 |

---

# ✔️ Po co to robimy?

Cross-walidacja:

* daje **stabilniejszy i mniej losowy wynik** niż jeden podział,
* lepiej pokazuje, jak model uogólnia na nowe dane,
* redukuje ryzyko, że wynik zależy od niefortunnego splitu.

---

# 🎯 Kiedy używać?

* gdy masz **niewiele danych**,
* gdy potrzebujesz **obiektywnej oceny modelu**,
* przy porównywaniu modeli (np. KNN vs. SVM vs. RandomForest),
* przy strojenia hiperparametrów (GridSearchCV, RandomizedSearchCV).

---

# 🧠 Ważne odmiany

| Rodzaj                    | Opis                                                  |
| ------------------------- | ----------------------------------------------------- |
| **Stratified K-fold**     | Zachowuje proporcje klas (idealne dla klasyfikacji).  |
| **Leave-One-Out (LOOCV)** | Każdy element raz jest testem (bardzo kosztowne).     |
| **TimeSeriesSplit**       | Do danych szeregów czasowych — nie miesza kolejności! |

---

# 📌 Mini schemat (intuicyjnie)

```
DANE → [train/test] x K → wyniki → średnia → ocena modelu
```




In [ ]:
## praktyczny przykład cross-validacji** w scikit-learn.


1. **Klasyczna K-fold cross-validation**
2. **Stratified K-fold** (standard przy klasyfikacji)

---

# ✅ 1. Klasyczna K-fold cross-validation



In [27]:
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Dane
X, y = load_iris(return_X_y=True)

# Model
model = RandomForestClassifier(random_state=42)

# K-fold: 5 podziałów
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Cross validation
scores = cross_val_score(model, X, y, cv=kf, scoring='accuracy')

print("Wyniki pojedynczych foldów:", scores)
print("Średnia:", scores.mean())
print("Odchylenie standardowe:", scores.std())

Wyniki pojedynczych foldów: [1.         0.96666667 0.93333333 0.93333333 0.96666667]
Średnia: 0.9600000000000002
Odchylenie standardowe: 0.024944382578492935


# 🎯 2. STRATIFIED K-fold (zalecane dla klasyfikacji)

W klasyfikacji proporcje klas muszą być zachowane w każdym foldzie — dlatego **StratifiedKFold** jest domyślnie używany przez `cross_val_score` przy klasyfikacji.

In [28]:
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression

X, y = load_iris(return_X_y=True)

model = LogisticRegression(max_iter=1000)

# Stratified K-fold
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

scores = cross_val_score(model, X, y, cv=skf, scoring='accuracy')

print("Wyniki foldów:", scores)
print("Średnia:", scores.mean())

Wyniki foldów: [1.         0.96666667 0.93333333 1.         0.93333333]
Średnia: 0.9666666666666668


In [ ]:
# 📌 3. Najkrótsza możliwa wersja (automatyczna CV)

Tu nie podajesz KFold — scikit-learn zrobi stratified K=5 domyślnie:

```python
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

X, y = load_iris(return_X_y=True)

model = SVC()

scores = cross_val_score(model, X, y, cv=5)

print(scores, scores.mean())
```

---

# ⚙️ 4. Cross-validacja w regresji

Tu zwykły KFold (bo nie ma klas):

```python
from sklearn.model_selection import KFold, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.datasets import load_diabetes

X, y = load_diabetes(return_X_y=True)

kf = KFold(n_splits=5, shuffle=True, random_state=42)

model = LinearRegression()

scores = cross_val_score(model, X, y, cv=kf, scoring="r2")

print(scores, scores.mean())
```

---

Jeśli chcesz, mogę też pokazać:

* GridSearchCV z cross-validacją,
* cross-validację w TimeSeriesSplit,
* porównanie modeli (np. RF vs SVM vs XGBoost) jednym kodem.